In [6]:
import pandas as pd
import numpy as np

In [7]:
# STEP 1 - GATHERING DATA


seasons_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MSeasons.csv')
#print(seasons_df.head())

conferences_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/Conferences.csv')
#print(conferences_df.head())

team_conferences_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MTeamConferences.csv')
#print(team_conferences_df.head())

tourney_seeds_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneySeeds.csv')
#print(tourney_seeds_df.head())

tourney_slots_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneySlots.csv')
#print(tourney_slots_df.head())

tourney_seed_round_slots_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneySeedRoundSlots.csv')
#print(tourney_seed_round_slots_df.head())

regseason_compact_results_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
#print(regseason_compact_results_df.head())

tourney_compact_results_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
#print(tourney_compact_results_df.head())

regseason_detailed_results_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
#print(regseason_detailed_results_df.head())

tourney_detailed_results_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
#print(tourney_detailed_results_df.head())

conference_tourney_games_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MConferenceTourneyGames.csv')
#print(conference_tourney_games_df.head())

massey_ordinals_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MMasseyOrdinals.csv')
#print(massey_ordinals_df.head())

sample_submission_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MSampleSubmissionStage1.csv')
#print(sample_submission_df.head())


               ID  Pred
0  2015_1107_1112   0.5
1  2015_1107_1116   0.5
2  2015_1107_1124   0.5
3  2015_1107_1125   0.5
4  2015_1107_1129   0.5


In [8]:
#STEP 2 - DATA PREPARATION

#Part 1 - Obtain TeamID, Season, Win %, Games Played, and PPG

#Find the game duration
regseason_compact_results_df['GameDuration'] = regseason_compact_results_df['NumOT']

# Normalize data for games with overtime
for i in range(len(regseason_compact_results_df.NumOT.value_counts())):
    regseason_compact_results_df.loc[regseason_compact_results_df['NumOT'] == regseason_compact_results_df.NumOT.value_counts().index[i], 'GameDuration'] = 40 + i*5

regseason_compact_results_df['WPPG'] = regseason_compact_results_df['WScore'].as_type(float) * (40.0 / regseason_compact_results_df['GameDuration'].as_type(float))
regseason_compact_results_df['LPPG'] = regseason_compact_results_df['LScore'].as_type(float) * (40.0 / regseason_compact_results_df['GameDuration'].as_type(float))

regseason_compact_results_df = regseason_compact_results_df.drop(['DayNum', 'WLoc', 'NumOT', 'WScore','LScore','GameDuration'], axis = 1)

#Calculate average points per game and number of games for winning and losing team
WPPG_df = regseason_compact_results_df[["Season","WTeamID",'WPPG']].groupby(["Season","WTeamID"]).agg(['mean', 'count'])
LPPG_df = regseason_compact_results_df[["Season","LTeamID",'LPPG']].groupby(["Season","LTeamID"]).agg(['mean', 'count'])

#Merge PPG for wins and losses and fill NaNs with 0s for undefeated and no-win teams
ppg_df = pd.merge(WPPG_df, LPPG_df, left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'], how='outer')
ppg_df = ppg_df.fillna({'WTeamID': ppg_df.LTeamID, 'WPPG': 0, 'WGames': 0, 'LTeamID': ppg_df.WTeamID , 'LPPG': 0, 'LGames': 0})

#Calculate PPG and win percentage for season
ppg_df['PPG'] = (ppg_df['WPPG']*ppg_df['WGames'] + ppg_df['LPPG']*ppg_df['LGames'])/(ppg_df['WGames'] + ppg_df['LGames'])
ppg_df['WPerc'] = (ppg_df['WGames'])/(ppg_df['WGames'] + ppg_df['LGames'])
ppg_df['TeamID'] = ppg_df['WTeamID'].astype('int32')
ppg_df['GamesPlayed'] = (ppg_df['WGames'] + ppg_df['LGames'])
ppg_df = ppg_df.drop(['WTeamID', 'WPPG' ,'WGames','LTeamID', 'LPPG','LGames'],axis = 1)

#Part 2 - Obtain game level statistics:
    #2-PT field goal efficiency (2-PT field goals made per game / 2-PT field goals attempted per game)
    #3-PT field goal efficiency
    #Total field goal efficiency
    #Free throw efficiency
    #Assists per game
    #Blocks per game
    #Steals per game
    #Turnovers per game
    #Personal fouls per game
    #Offensive rebounds per game
    #Defensive rebounds per game
    #Points per shot
    #Total rebounds per game
    #Average turnovers to assists ratio
    #Average turnovers to steals ratio
    #Average scoring margin


#Gametime normalization
regseason_detailed_results_df['GameDuration'] = regseason_detailed_results_df['NumOT']
for i in range(len(regseason_detailed_results_df.NumOT.value_counts())):
    regseason_detailed_results_df.loc[regseason_detailed_results_df['NumOT'] == regseason_detailed_results_df.NumOT.value_counts().index[i], 'GameDuration'] = 40 + i*5

regseason_detailed_results_df['WFGM'] = (regseason_detailed_results_df['WFGM'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFGA'] = (regseason_detailed_results_df['WFGA'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFGM3'] = (regseason_detailed_results_df['WFGM3'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFGA3'] = (regseason_detailed_results_df['WFGA3'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFTM'] = (regseason_detailed_results_df['WFTM'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFTA'] = (regseason_detailed_results_df['WFTA'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WOR'] = (regseason_detailed_results_df['WOR'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WDR'] = (regseason_detailed_results_df['WDR'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WAst'] = (regseason_detailed_results_df['WAst'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WTO'] = (regseason_detailed_results_df['WTO'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WStl'] = (regseason_detailed_results_df['WStl'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WBlk'] = (regseason_detailed_results_df['WBlk'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WPF'] = (regseason_detailed_results_df['WPF'] / regseason_detailed_results_df['GameDuration']) * 40

regseason_detailed_results_df['LFGM'] = (regseason_detailed_results_df['LFGM'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFGA'] = (regseason_detailed_results_df['LFGA'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFGM3'] = (regseason_detailed_results_df['LFGM3'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFGA3'] = (regseason_detailed_results_df['LFGA3'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFTM'] = (regseason_detailed_results_df['LFTM'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFTA'] = (regseason_detailed_results_df['LFTA'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LOR'] = (regseason_detailed_results_df['LOR'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LDR'] = (regseason_detailed_results_df['LDR'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LAst'] = (regseason_detailed_results_df['LAst'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LTO'] = (regseason_detailed_results_df['LTO'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LStl'] = (regseason_detailed_results_df['LStl'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LBlk'] = (regseason_detailed_results_df['LBlk'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LPF'] = (regseason_detailed_results_df['LPF'] / regseason_detailed_results_df['GameDuration']) * 40

#Calculate game level statistics for winning team

regseason_detailed_results_df["W2FG%"] = (regseason_detailed_results_df["WFGM"] / regseason_detailed_results_df["WFGA"])
regseason_detailed_results_df["W3FG%"] = (regseason_detailed_results_df["WFGM3"] / regseason_detailed_results_df["WFGA3"])
regseason_detailed_results_df["WFG%"] = (regseason_detailed_results_df["WFGM"] + regseason_detailed_results_df["WFGM3"] ) / (regseason_detailed_results_df["WFGA"] + regseason_detailed_results_df["WFGA3"])
regseason_detailed_results_df["WFT%"] = (regseason_detailed_results_df['WFTM'] / regseason_detailed_results_df['WFTA'])
regseason_detailed_results_df["WScoreMargin"] = regseason_detailed_results_df["WScore"] - regseason_detailed_results_df["LScore"]
regseason_detailed_results_df["WPPS"] = (regseason_detailed_results_df["WScore"]) / (regseason_detailed_results_df["WFGA"] + regseason_detailed_results_df["WFGA3"])
regseason_detailed_results_df["WOEFF"] = (regseason_detailed_results_df["WScore"] / ((regseason_detailed_results_df["WFGA"] + regseason_detailed_results_df["WFGA3"]) - regseason_detailed_results_df['WOR'] + regseason_detailed_results_df['WTO'] + (0.4 x regseason_detailed_results_df['WFTA'])))
regseason_detailed_results_df["WDEFF"] = (regseason_detailed_results_df["LScore"] / ((regseason_detailed_results_df["WFGA"] + regseason_detailed_results_df["WFGA3"]) - regseason_detailed_results_df['WOR'] + regseason_detailed_results_df['WTO'] + (0.4 x regseason_detailed_results_df['WFTA'])))

#Calculate game level statistics for losing team

regseason_detailed_results_df["L2FG%"] = (regseason_detailed_results_df["LFGM"] / regseason_detailed_results_df["LFGA"])
regseason_detailed_results_df["L3FG%"] = (regseason_detailed_results_df["LFGM3"] / regseason_detailed_results_df["LFGA3"])
regseason_detailed_results_df["LFG%"] = (regseason_detailed_results_df["LFGM"] + regseason_detailed_results_df["LFGM3"] ) / (regseason_detailed_results_df["LFGA"] + regseason_detailed_results_df["LFGA3"])
regseason_detailed_results_df["LFT%"] = (regseason_detailed_results_df['LFTM'] / regseason_detailed_results_df['LFTA'])
regseason_detailed_results_df["LScoreMargin"] = (regseason_detailed_results_df["LScore"] - regseason_detailed_results_df["WScore"])
regseason_detailed_results_df["LPPS"] = (regseason_detailed_results_df["LScore"]) / (regseason_detailed_results_df["LFGA"] + regseason_detailed_results_df["LFGA3"])
regseason_detailed_results_df["LOEFF"] = (regseason_detailed_results_df["LScore"] / ((regseason_detailed_results_df["LFGA"] + regseason_detailed_results_df["LFGA3"]) - regseason_detailed_results_df['LOR'] + regseason_detailed_results_df['LTO'] + (0.4 x regseason_detailed_results_df['LFTA'])))
regseason_detailed_results_df["LDEFF"] = (regseason_detailed_results_df["WScore"] / ((regseason_detailed_results_df["LFGA"] + regseason_detailed_results_df["LFGA3"]) - regseason_detailed_results_df['LOR'] + regseason_detailed_results_df['LTO'] + (0.4 x regseason_detailed_results_df['LFTA'])))


#Take seasonal averages of the season level and game level metrics
#WPPG_df = regseason_compact_results_df[["Season","WTeamID",'WPPG']].groupby(["Season","WTeamID"]).agg(['mean', 'count'])
#LPPG_df = regseason_compact_results_df[["Season","LTeamID",'LPPG']].groupby(["Season","LTeamID"]).agg(['mean', 'count'])
game_details_df =

SyntaxError: invalid syntax (<ipython-input-8-a6141b927931>, line 90)

In [ ]:
#Part 4 - Add Ranking Data
rankings_df = massey_df[massey_df['RankingDayNum'] == 133]
rankings_df = rankings_df.reset_index().drop(['index','RankingDayNum'], axis = 1)
mean_df = rankings_df.groupby(by = ['Season','TeamID'])[['OrdinalRank']].mean().reset_index()
mean_df = mean_df.rename(columns = {'OrdinalRank':'MeanRank'})
massey_df = rankings_df[rankings_df['SystemName'] == 'MAS']
massey_df = massey_df.reset_index().drop(['index','SystemName'], axis = 1)
massey_df = massey_df.rename(columns = {'OrdinalRank':'MasseyRank'})
pomeroy_df = rankings_df[rankings_df['SystemName'] == 'POM']
pomeroy_df = pomeroy_df.reset_index().drop(['index','SystemName'], axis = 1)
pomeroy_df = pomeroy_df.rename(columns = {'OrdinalRank':'PomeroyRank'})
rankings_df = pd.merge(massey_df, pomeroy_df, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')
rankings_df = pd.merge(rankings_df, mean_df, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')

In [ ]:
#Part 5 - Merge the dataframes into one feature set

#Create feature list for each season of every D1 team
feature_names = [ "TeamID", "Season", "Win %", "Games Played", "PPG", "2FG%", "3FG%","FG%", "FT%", "APG", "BPG", "STLPG", "TPG", "PFPG", "ORPG", "DRPG", "PPS", "TRPG", "TAR","TSR", "ASM", "AvgRank", "MRank", "PRank"]
features_df = pd.DataFrame(columns = feature_names)

In [ ]:
# Feature selection and reduction


In [ ]:
#STEP 3 - CHOOSE MODELS


In [ ]:

#STEP 4 - TRAIN MODEL


In [ ]:

#STEP 5 - EVALUATE THE MODEL


In [ ]:

#STEP 6 - TUNE THE PARAMETERS


In [ ]:

#STEP 7 - CREATE FINAL PREDICTIONS FOR 2021 TOURNAMENT

